In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd

In [2]:
class Net(nn.Module):

    # Model to regress 2d time series values given scalar input.
    def __init__(self):
        super(Net, self).__init__()
        # define two fully connected layers with 32 hidden units each
        self.fc1 = nn.Linear(1, 32)
        self.fc2 = nn.Linear(32, 32)
        # define the output layer with 2 units for the x and y values
        self.fc3 = nn.Linear(32, 2)

    def forward(self, x):
        # reshape the input tensor to have the shape (batch_size, 1)
        x = x.view(-1, 1)
        # apply the ReLU activation function to the output of the first two layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # return the output of the third layer without applying any activation function
        return self.fc3(x)

In [3]:
class TimeSeriesDataset(torch.utils.data.Dataset):

    def __init__(self, csv_file):
        # load the data from the CSV file and replace '-' with 0
        self.data = pd.read_csv(csv_file).replace('-','0')
        # convert the x and y columns to numbers
        self.data[['x', 'y']] = self.data[['x', 'y']].astype(float)

    def __len__(self):
        # return the length of the dataset
        return len(self.data)

    def __getitem__(self, idx):
        # return a tuple containing the input time and the corresponding x and y values as a tensor
        time = self.data.iloc[idx, 0]
        x = self.data.iloc[idx, 1]
        y = self.data.iloc[idx, 2]
        time_tensor = torch.tensor(time, dtype=torch.float).unsqueeze(0)
        labels = torch.tensor((x, y), dtype=torch.float)
        return (time_tensor, labels)

In [4]:
def loss_fn(outputs, labels):
    # compute the mean squared error loss
    return torch.mean((outputs - labels) ** 2)


BATCH_SIZE = 128

# initialize the dataset and data loader
dataset = TimeSeriesDataset('data.csv')
trainloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

# initialize the model, loss function, and optimizer
net = Net()
optimizer = optim.Adam(net.parameters())

In [5]:
for epoch in range(300):

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 20 == 19:    # print every 20 mini-batches
            print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 20))
            running_loss = 0.0

print('Finished Training')

Finished Training


In [6]:
# Plot results to validate training